In [3]:
import os
import time
from openai import OpenAI
import streamlit as st
from dotenv import load_dotenv
import json
import time

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI")

In [4]:
client = OpenAI(api_key=OPENAI_API_KEY)

assistant = client.beta.assistants.create(
    name = "Finance Insight Analyst",
    instructions = "You are a helpful financial analyst expert and, focusing on management discussions and financial results. Help people learn about financial needs and guide them towards fincial literacy.",
    tools = [{"type":"code_interpreter"}, {"type": "retrieval"}],
    model = "gpt-4-1106-preview"
)

In [11]:
def return_json(obj):
    return json.loads(obj.model_dump_json())

assitant_desc = return_json(assistant)

In [13]:
assitant_desc.keys()

dict_keys(['id', 'created_at', 'description', 'file_ids', 'instructions', 'metadata', 'model', 'name', 'object', 'tools'])

In [8]:
thread = client.beta.threads.create()

In [10]:
def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )

In [11]:
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [12]:
def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

In [13]:
def pretty_print(messages):
    responses = []
    for m in messages:
        if m.role == "assistant":
            responses.append(m.content[0].text.value)
    return "\n".join(responses)